In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Install required packages
!pip install kaggle scikit-learn

# Import libraries
import sqlite3
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Download Kaggle dataset
try:
    !kaggle datasets download -d pavansubhasht/ibm-hr-analytics-attrition-dataset
    !unzip -o ibm-hr-analytics-attrition-dataset.zip
except Exception as e:
    print(f"Error downloading dataset: {e}")
    print("Ensure Kaggle API is configured and internet is active.")
    raise

# Load data into pandas
try:
    df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv', encoding='latin1')
    print("Dataset Columns:", df.columns.tolist())  # Debug: Check column names
except FileNotFoundError:
    print("Error: WA_Fn-UseC_-HR-Employee-Attrition.csv not found. Check Kaggle download.")
    raise
except Exception as e:
    print(f"Error loading CSV file: {e}")
    raise

# Rename 'ï»¿Age' to 'Age' to match schema
df.rename(columns={'ï»¿Age': 'Age'}, inplace=True)

# Verify required columns (after rename)
required_columns = ['EmployeeNumber', 'Department', 'JobRole', 'MonthlyIncome', 'PerformanceRating', 'YearsAtCompany', 'Attrition', 'Age', 'JobSatisfaction', 'JobLevel']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"Error: Missing columns {missing_columns}. Available columns: {df.columns.tolist()}")
    raise KeyError(f"Missing columns: {missing_columns}")

# Connect to SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create SQLite table with constraints (Database Concepts)
cursor.execute('''
CREATE TABLE Employees (
    EmployeeNumber INTEGER PRIMARY KEY,
    Department TEXT NOT NULL,
    JobRole TEXT,
    MonthlyIncome REAL,
    PerformanceRating INTEGER,
    YearsAtCompany INTEGER,
    Attrition TEXT,
    Age INTEGER,
    JobSatisfaction INTEGER,
    JobLevel INTEGER
)
''')

# Insert data into SQLite
df[required_columns].to_sql('Employees', conn, if_exists='append', index=False)

# Demonstrate INSERT (Basic SQL Syntax)
# Insert a new employee
cursor.execute('''
INSERT INTO Employees (EmployeeNumber, Department, JobRole, MonthlyIncome, PerformanceRating, YearsAtCompany, Attrition, Age, JobSatisfaction, JobLevel)
VALUES (9999, 'Sales', 'Sales Executive', 5000.0, 3, 2, 'No', 30, 3, 2)
''')

# Demonstrate UPDATE (Basic SQL Syntax)
# Update PerformanceRating for long-tenure employees
cursor.execute('''
UPDATE Employees
SET PerformanceRating = 4
WHERE YearsAtCompany > 10 AND PerformanceRating < 4
''')

# Demonstrate DELETE (Basic SQL Syntax)
# Delete employees with low JobSatisfaction
cursor.execute('''
DELETE FROM Employees
WHERE JobSatisfaction < 2
''')

# SQL Queries to demonstrate skills and provide insights
queries = [
    # Query 1: Department performance stats (Retrieve, Manipulate)
    '''
    SELECT
        Department,
        COUNT(*) as employee_count,
        ROUND(AVG(MonthlyIncome), 2) as avg_income,
        AVG(PerformanceRating) as avg_performance
    FROM Employees
    GROUP BY Department
    ORDER BY avg_performance DESC;
    ''',

    # Query 2: Top performers by job role with subquery (Complex Query)
    '''
    SELECT
        JobRole,
        EmployeeNumber,
        PerformanceRating,
        MonthlyIncome,
        (SELECT AVG(PerformanceRating) FROM Employees e2 WHERE e2.JobRole = e1.JobRole) as avg_role_rating
    FROM Employees e1
    WHERE PerformanceRating > (SELECT AVG(PerformanceRating) FROM Employees e3 WHERE e3.JobRole = e1.JobRole)
    ORDER BY JobRole, PerformanceRating DESC
    LIMIT 5;
    ''',

    # Query 3: Attrition risk by tenure with window function (Complex Query)
    '''
    SELECT
        YearsAtCompany,
        COUNT(*) as employee_count,
        SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) as attrition_count,
        ROUND(SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as attrition_rate,
        AVG(PerformanceRating) OVER (PARTITION BY YearsAtCompany) as avg_rating
    FROM Employees
    GROUP BY YearsAtCompany
    ORDER BY attrition_rate DESC
    LIMIT 5;
    ''',

    # Query 4: Departmental income disparity with CTE (Complex Query)
    '''
    WITH IncomeStats AS (
        SELECT
            Department,
            MonthlyIncome,
            DENSE_RANK() OVER (PARTITION BY Department ORDER BY MonthlyIncome DESC) as income_rank
        FROM Employees
    )
    SELECT
        Department,
        MAX(CASE WHEN income_rank = 1 THEN MonthlyIncome END) as top_income,
        MIN(MonthlyIncome) as bottom_income,
        ROUND((MAX(CASE WHEN income_rank = 1 THEN MonthlyIncome END) - MIN(MonthlyIncome)) / MIN(MonthlyIncome) * 100, 2) as disparity_percent
    FROM IncomeStats
    GROUP BY Department;
    ''',

    # Query 5: Job satisfaction trends by age group (Complex Query)
    '''
    SELECT
        age_group,
        COUNT(*) as employee_count,
        ROUND(AVG(JobSatisfaction), 2) as avg_satisfaction,
        SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) as attrition_count
    FROM (
        SELECT
            CASE
                WHEN Age < 30 THEN 'Under 30'
                WHEN Age BETWEEN 30 AND 40 THEN '30-40'
                ELSE 'Over 40'
            END as age_group,
            JobSatisfaction,
            Attrition
        FROM Employees
    ) sub
    GROUP BY age_group
    ORDER BY avg_satisfaction DESC;
    ''',

    # Query 6: Performance consistency with CTE and window function (Complex Query)
    '''
    WITH PerformanceStats AS (
        SELECT
            Department,
            PerformanceRating,
            YearsAtCompany,
            ROW_NUMBER() OVER (PARTITION BY Department ORDER BY PerformanceRating DESC) as rank
        FROM Employees
    )
    SELECT
        Department,
        COUNT(*) as total_employees,
        AVG(PerformanceRating) as avg_rating,
        MAX(CASE WHEN rank = 1 THEN PerformanceRating END) as top_rating,
        SUM(CASE WHEN PerformanceRating >= 4 THEN 1 ELSE 0 END) as high_performers
    FROM PerformanceStats
    GROUP BY Department
    ORDER BY avg_rating DESC;
    '''
]

# Execute queries and export to CSV for Excel
for i, query in enumerate(queries, 1):
    print(f"\nQuery {i} Results:")
    try:
        df_result = pd.read_sql_query(query, conn)
        print(df_result)
        # Export to CSV for Excel analysis
        df_result.to_csv(f'employee_query_{i}.csv', index=False)
        print(f"Exported Query {i} results to employee_query_{i}.csv")
    except Exception as e:
        print(f"Error executing query {i}: {e}")

# Machine Learning: Predict High Performers
# Prepare data for ML
ml_df = pd.read_sql_query("SELECT * FROM Employees", conn)
features = ['MonthlyIncome', 'YearsAtCompany', 'Age', 'JobSatisfaction', 'JobLevel']
X = ml_df[features]
# Create target: HighPerformer (1 if PerformanceRating >= 4, 0 otherwise)
y = (ml_df['PerformanceRating'] >= 4).astype(int)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model with class balancing
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("\nMachine Learning - High Performer Prediction Results:")
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Not High Performer', 'High Performer']))

# Export predictions for Excel analysis
predictions_df = pd.DataFrame({
    'Actual': ['High Performer' if x == 1 else 'Not High Performer' for x in y_test],
    'Predicted': ['High Performer' if x == 1 else 'Not High Performer' for x in y_pred]
})
predictions_df.to_csv('high_performer_predictions.csv', index=False)
print("Exported ML predictions to high_performer_predictions.csv")

# Clean up
try:
    os.remove('ibm-hr-analytics-attrition-dataset.zip')
    os.remove('WA_Fn-UseC_-HR-Employee-Attrition.csv')
except:
    pass
conn.close()

Dataset URL: https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset
License(s): DbCL-1.0
Archive:  ibm-hr-analytics-attrition-dataset.zip
  inflating: WA_Fn-UseC_-HR-Employee-Attrition.csv  
Dataset Columns: ['ï»¿Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

Query 1 Results:
               Department  employee_count  avg_income  avg_performance
0  Research & Development             769     6258.09  